In [2]:
import datetime
import jieba
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
zhihu_question = pd.read_csv(r'E:\pythonProject\制作趋势词典\知乎问题数据集.csv',encoding='utf-8',header=0,index_col=0,sep=r',',engine='python')
zhihu_question

,0
0,12 月 30 日武汉出现不明原因肺炎，目前情况如何？
1,2019 年，你的「至暗时刻」是什么时候？
2,大学生活有个讨厌的舍友，又碍于面子没法撕逼是怎样一种体验？
3,全职妈妈最终真的会成为悲剧吗？
4,总结2019，展望2020！(附商誉地雷股名单)
...,...
598550,为什么那么焦虑？
598551,塞维中场：我对米兰非常感兴趣，从小就梦想为红黑军团效力
598552,华晨宇真的没有团队吗？为什么部分人坚定地认为华晨宇没有团队？
598553,详细的自我介绍


In [4]:
# zhihu_samples = zhihu_question.sample(10000)
zhihu = zhihu_question['0'].to_list()
len(zhihu)

598555

In [5]:
import pandas as pd
seed_words1 = pd.read_excel(r'E:\论文\数据\WenXinV4.0师姐发的版本\dic\dictionary_all.xlsx',header=0,index_col=None)
seed_words1
seed_words = seed_words1['words'].to_list()
len(seed_words)

12437

In [6]:
jieba.load_userdict(r'E:\pythonProject\需求分析词典\dictionary.txt')
def tokenize_zh(text):
    words = jieba.lcut(text)
    return words
vectorizer = CountVectorizer(tokenizer=tokenize_zh)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Shy0418\AppData\Local\Temp\jieba.cache
Loading model cost 0.675 seconds.
Prefix dict has been built successfully.


In [ ]:
# 这段不要随便跑
from keybert import KeyBERT
kw_model =  KeyBERT('all-MiniLM-L6-v2')
count = 0
path_keybert = r'Keybert抽取关键词.txt'
with open(path_keybert,mode='w+',encoding='utf-8') as file_keybert:
    for i in zhihu:
        keywords = kw_model.extract_keywords(i,vectorizer=vectorizer,stop_words='E:\论文\python代码\停用词表\stopwords_all.txt',keyphrase_ngram_range=[1,2],use_maxsum=False,use_mmr=False,top_n = 3,seed_keywords = seed_words)
        for j in range(len(keywords)):
            # print(j)
            file_keybert.write(str(keywords[j][0]+'\t'))
        file_keybert.write('\n')
        # print(keywords[0][0])
        # print(keywords[1][0])
        print(count)
        count+=1
# file_keybert.flush()
file_keybert.close()
time_end = datetime.datetime.now()

In [9]:
import csv
import pandas as pd
data = pd.read_table(r'E:\pythonProject\需求分析词典\Keybert抽取关键词.txt',encoding='utf-8',header=None,sep='\t',quoting=csv.QUOTE_NONE)
data1 = data.iloc[:,0:3]
data1

,0,1,2
0,原因,出现,武汉
1,至暗,时刻,时候
2,碍于面子,没法,生活
3,成为,真的,最终
4,名单,地雷,总结
...,...,...,...
598550,为什么,那么,焦虑
598551,中场,效力,从小
598552,坚定,没有,真的
598553,自我介绍,的,详细


In [12]:
import numpy as np
data2 = np.array(data1)
data2
data3 = data2.reshape(1795665,1)

pd.DataFrame(data3).to_csv(r'Keybert抽取关键词处理后.txt',encoding='utf-8',header=None,index=None)
len(data3)

1795665

In [1]:
import pandas as pd
import csv
stopwords = pd.read_table(r'E:\pythonProject\需求分析词典\stopwords_all.txt',encoding='utf-8',header=None,quoting=csv.QUOTE_NONE)
stopwords1 = stopwords[0].tolist()
len(stopwords1)

3887

In [ ]:
# 统计keybert抽取的关键词词频
txt = open(r'E:\pythonProject\制作趋势词典\Keybert抽取关键词处理后.txt', 'r',encoding='utf-8').readlines()    # 获取文本文件
counts = {}    # 创建空字典
for num in txt:
    if num == ' ':    # 排除数字文本中可能出现的空格
        continue
    else:
        counts[num] = counts.get(num, 0) + 1  # 统计词频并在字典中创建键值对
items = list(counts.items())       # 将无序的字典类型转换为可排序的列表类型
items.sort(key=lambda x: x[1], reverse=True)   # 以元素的第二列进行从大到小排序

for i in range(10000):
    num, count = items[i]
    print("{:<5}:{:>5}".format(num, count))    # 格式化输出排序结果
    # print(num)
    # print(type(num))
    with open(r'Keybert提取前10000个关键词.txt',mode='a+',encoding='utf-8') as file:
        if num not in stopwords1:
            file.write(num)
        else:
            continue
        # file.write('\n')
file.close()


In [4]:
# TF_IDF抽取
path_tf_idf = r'TF_IDF抽取关键词.txt'
import jieba  # 分词器
import jieba.analyse
import pandas as pd
jieba.load_userdict(r'E:\pythonProject\需求分析词典\dictionary.txt')
def tfidf_ana(content):
    title_keys = jieba.analyse.extract_tags(content, topK=1000000, withWeight=False)  # topK为期望得到的关键词个数
    with open(path_tf_idf, mode='w+',encoding='utf-8') as file:
        for i in title_keys:
            file.write(i + '\n')
        file.close()

In [ ]:
answer_df = pd.read_csv('E:\pythonProject\需求分析词典\知乎问题数据集.csv',encoding='utf-8')
answer_list = answer_df['0'].to_list()
answer_str = ''
i=0
for k in answer_list:
    print(i)
    i+=1
    answer_str += str(k)
    print(k)
data = tfidf_ana(answer_str)

In [8]:
# 判断是否是中文
def is_Chinese(word):#修改过的
    for ch in word:
        if '\u4e00' > ch or ch > '\u9fff':
            return False
    return True

In [6]:
KeyBert_keyword = pd.read_table(r'E:\pythonProject\需求分析词典\Keybert抽取关键词处理后.txt',encoding='utf-8',header=None)
Tf_IDF_keyword = pd.read_table(r'E:\pythonProject\需求分析词典\TF_IDF抽取关键词.txt',encoding='utf-8',header=None)
# # KeyBert_keyword
# Tf_IDF_keyword
KeyBert_Tfidf_keyworeds=pd.concat([KeyBert_keyword,Tf_IDF_keyword],axis=0)
KeyBert_Tfidf_keyworeds.drop_duplicates(subset=0,keep='first',inplace=True)
KeyBert_Tfidf_keyworeds

,0
0,原因
1,出现
2,武汉
3,至暗
4,时刻
...,...
177001,证券报
177003,连忙
177004,吩咐
177005,起身


In [9]:
# 仅保留中文
with open(r'去重后的关键词.txt',mode='w+',encoding='utf-8') as file:
    for i in range(len(KeyBert_Tfidf_keyworeds)):
        if(is_Chinese(str(KeyBert_Tfidf_keyworeds.iloc[i,0])) and len(str(KeyBert_Tfidf_keyworeds.iloc[i,0]))>1):
            file.write(str(KeyBert_Tfidf_keyworeds.iloc[i,0]))
            file.write('\n')
file.close()

In [10]:
# 以下测试同义词工具包
import synonyms
# synonyms.display('问题')
a = synonyms.nearby('问题',10)

Building prefix dict from d:\python3.8\lib\site-packages\synonyms\data\vocab.txt ...



 Synonyms: v3.18.0, Project home: https://github.com/chatopera/Synonyms/

 Project Sponsored by Chatopera

  deliver your chatbots with Chatopera Cloud Services --> https://bot.chatopera.com

>> Synonyms load wordseg dict [d:\python3.8\lib\site-packages\synonyms\data\vocab.txt] ... 


Dumping model to file cache C:\Users\Shy0418\AppData\Local\Temp\jieba.u4ffff3a26e04aaea5c89ddb932e881e1.cache
Loading model cost 1.406 seconds.
Prefix dict has been built successfully.


>> Synonyms on loading stopwords [d:\python3.8\lib\site-packages\synonyms\data\stopwords.txt] ...
>> Synonyms on loading vectors [d:\python3.8\lib\site-packages\synonyms\data\words.vector.gz] ...


In [11]:
import numpy as np
np.array(a).T

array([['问题', '1.0'],
       ['难题', '0.6956479'],
       ['缺陷', '0.63884103'],
       ['弊端', '0.638318'],
       ['环境问题', '0.624778'],
       ['结构性问题', '0.613208'],
       ['关键问题', '0.6061078'],
       ['弊病', '0.60467'],
       ['症结', '0.5877637'],
       ['风险问题', '0.5795508']], dtype='<U32')